# A class for turtle graphics in 3D using VPython

In [15]:
from vpython import *
from LSystems import *

class Turtle3D:
    def __init__(self, pos=vector(0,0,0), heading = vector(0,1,0), leftDirection = vector(0,0,1), upDirection = vector(1,0,0), width = 1):
        self.position        = pos
        self.heading         = heading
        self.leftDirection   = leftDirection
        self.upDirection     = upDirection
        self.isDrawing       = True
        self.curveStack      = []
        self.widthStack      = []
        self.colorStack      = []
        self.listOfVectors   = []
        self.c               = curve(self.position)
        self.width           = width
        self.polygon         = False
        self.currentColor    = vector(51/255,25/255,0)
        
    def forward(self, distance = 1):
        oldPosition = self.position
        self.position = self.position + distance * norm(self.heading)
        keys = ['pos', 'color' ,'radius']
        values = [self.position, self.currentColor, self.width]
        new_pos = dict(zip(keys, values))
        if self.isDrawing:
            self.c.append(new_pos)
        if self.polygon:
            self.listOfVectors.append(self.position)
    
    def turnLeft(self, angle = 22.5):
        self.leftDirection = self.leftDirection.rotate(radians(angle),self.upDirection)
        self.heading  = self.heading.rotate(radians(angle),self.upDirection)
    
    def turnRight(self, angle = 22.5):
        self.leftDirection = self.leftDirection.rotate(radians(-angle),self.upDirection)
        self.heading  = self.heading.rotate(radians(-angle),self.upDirection)
    
    def pitchUp(self, angle = 22.5):
        self.upDirection = self.upDirection.rotate(radians(angle),self.leftDirection)
        self.heading  = self.heading.rotate(radians(angle),self.leftDirection)
    
    def pitchDown(self, angle = 22.5):
        self.upDirection = self.upDirection.rotate(radians(-angle),self.leftDirection)
        self.heading  = self.heading.rotate(radians(-angle),self.leftDirection)
    
    def rollLeft(self, angle = 22.5):
        self.upDirection = self.upDirection.rotate(radians(angle),self.heading)
        self.leftDirection  = self.leftDirection.rotate(radians(angle),self.heading)
    
    def rollRight(self, angle = 22.5):
        self.upDirection = self.upDirection.rotate(radians(-angle),self.heading)
        self.leftDirection  = self.leftDirection.rotate(radians(-angle),self.heading)
        
    def xcor(self):
        return(self.position.x)
    
    def ycor(self):
        return(self.position.y)
    
    def zcor(self):
        return(self.position.z)
    
    def penUp(self):
        self.isDrawing = False
    
    def penDown(self):
        self.isDrawing = True
        # Start a new curve when we restart drawing
        keys = ['pos', 'color' ,'radius']
        values = [self.position, self.currentColor, self.width]
        new_pos = dict(zip(keys, values))
        self.c = curve(new_pos)
    
    def setHeading(self,heading):
        self.heading       = heading[0]
        self.leftDirection = heading[1]
        self.upDirection   = heading[2]
    
    def getHeading(self):
        return((self.heading, self.leftDirection, self.upDirection))
    
    def setPosition(self, pos):
        self.position = pos
    
    def getPosition(self):
        return(self.position)
    
    def pushCurve(self):
        self.curveStack.insert(0, self.c)
        self.widthStack.insert(0, self.width)
        self.colorStack.insert(0, self.currentColor)
        # start new curve
        keys = ['pos', 'color', 'radius']
        values = [self.position, self.currentColor, self.width]
        new_pos = dict(zip(keys, values))
        self.c = curve(new_pos)
    
    def popCurve(self):
        self.c = self.curveStack.pop(0)
        self.width = self.widthStack.pop(0)
        self.currentColor = self.colorStack.pop(0)
        
    def decrementDiameter(self, scaling):
        self.width = self.width*scaling
        
    def startPolygon(self):
        """Only works for convex polygons in a single plane."""
        self.polygon = True
        self.listOfVectors.append(self.position)
    
    def endPolygon(self):
        """Only works for convex polygons in a single plane.
        
        This is just simple fan triangulation
        """
        self.polygon = False
        for i in range(2,len(self.listOfVectors)):
            triangle(vs = [vertex(pos = self.listOfVectors[0], color = self.currentColor), 
                           vertex(pos = self.listOfVectors[i-1],color = self.currentColor), 
                           vertex(pos = self.listOfVectors[i],color = self.currentColor)])
        self.listOfVectors = []
        
    def setColor(self, rgb):
        self.currentColor = vector(rgb[0]/255,rgb[1]/255,rgb[2]/255)
    

def turtle_interpretation_3D(instructions, delta = 22.5, width = 0.3, widthScaling = 0.7):
    """Interprets a set of instructions for a turtle to draw a tree in 3D. """    
    ############ INITIALIZATION #############
    scene = canvas() # This is needed in Jupyter notebook and lab to make programs easily rerunnable
    bob = Turtle3D(width = width)
    
    turtlePosStack = []
    turtleHeadStack = []
    
    turtleXmax = 0
    turtleXmin = 0
    turtleYmax = 0 
    turtleYmin = 0
    turtleZmax = 0
    turtleZmin = 0
    
    ######### MAIN FUNCTION ##############
    for mod in instructions:
        if mod.symbol == "F":
            bob.forward()
            #Note that we only need to update the max coordinates if the turtle has moved
            turtleXmax = max(turtleXmax, bob.xcor())
            turtleXmin = min(turtleXmin, bob.xcor())
            turtleYmax = max(turtleYmax, bob.ycor())
            turtleYmin = min(turtleYmin, bob.ycor())
            turtleZmax = max(turtleZmax, bob.zcor())
            turtleZmin = min(turtleZmin, bob.zcor()) 
        elif mod.symbol == "f":
            bob.penUp()
            bob.forward()
            bob.penDown()
        elif mod.symbol == "+":
            bob.turnLeft(delta)
        elif mod.symbol == "-":
            bob.turnRight(delta)
        elif mod.symbol == "&":
            bob.pitchDown(delta)
        elif mod.symbol == "^":
            bob.pitchUp(delta)
        elif mod.symbol == "\\":
            bob.rollLeft(delta)
        elif mod.symbol == "/":
            bob.rollRight(delta)
        elif mod.symbol == "|":
            bob.turnLeft(180)
        elif mod.symbol == "[":
            turtlePosStack.insert(0,bob.getPosition())
            turtleHeadStack.insert(0,bob.getHeading())
            bob.pushCurve()
        elif mod.symbol == "]":
            bob.setPosition(turtlePosStack.pop(0))
            bob.setHeading(turtleHeadStack.pop(0)) 
            bob.popCurve()
        elif mod.symbol == "!":
            bob.decrementDiameter(widthScaling)
        elif mod.symbol == "{":
            bob.startPolygon()
        elif mod.symbol == "}":
            bob.endPolygon()
        elif mod.symbol == "'":
            bob.setColor(mod.param)
    
    #point the camera to the center of our drawing
    xcenter = (turtleXmax+turtleXmin)/2
    ycenter = (turtleYmax+turtleYmin)/2
    zcenter = (turtleZmax+turtleZmin)/2
    scene.center=vector(xcenter,ycenter,zcenter)
    scene.ambient=color.gray(0.4)
    return 0
    

        

## 3D turtle graphics applied to a simple L-system

In [18]:
########## INPUT #########
axiom = "A"    
productions = {"A?'(151,75,0) [ & F L ! A ] / / / / / '(151,75,0) [ & F L ! A ] / / / / / / / '(151,75,0) [ & F L ! A ]", 
               "F?S / / / / / F",
               "S?F L",
               "L?[ '(12,102,0) ^ ^ { - f + f + f - | - f + f + f } ]"} 
nrOfIterations = 7

########### MAIN #############

# Initialize
system = LSystem(axiom,productions)

for i in range(nrOfIterations):
    tree = system.nextGeneration()

turtle_interpretation_3D(tree, delta = 22.5, width = 0.5, widthScaling = 0.6)

<IPython.core.display.Javascript object>

0